# Interruption du fou

La dernière fois que vous avez vu ces textes de Nietzsche, vous les avez quittés avec une matrice d’occurrences sans ne rien en faire de plus. Il est temps de la transformer en une matrice termes-documents pondérée par l’algorithme TF-IDF.

Exécutez ci-dessous le code afin de créer une matrice des occurrences :

In [ ]:
import numpy as np
import pandas as pd
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

# a corpus of documents
corpus = [
    "Tous les hommes qui sentent qu’il leur faut les paroles et les intonations les plus violentes, les attitudes et les gestes les plus éloquents pour pouvoir agir, les politiciens révolutionnaires, les socialistes, les prédicateurs, avec ou sans christianisme, tous ceux qui veulent éviter les demi-succès : tous ceux-là parlent de devoirs, et toujours de devoirs qui ont un caractère absolu - autrement ils n’auraient point droit à leur pathos démesuré : ils le savent fort bien.",
    "Il faut connaître non seulement la marche hardie, légère, délicate et rapide de ses propres pensées, mais avant tout la disposition aux grandes responsabilités, la hauteur et la profondeur du regard impérieux, le sentiment d’être séparé de la foule, des devoirs et des vertus de la foule, la protection et la défense bienveillante de ce qui est mal compris et calomnié, que ce soit Dieu ou le diable ; le penchant et l’habileté à la suprême justice, l’art du commandement, l’ampleur de la volonté, la lenteur du regard qui rarement admire, rarement se lève, et aime rarement…",
    "Je vous le dis : il faut encore porter en soi un chaos, pour pouvoir mettre au monde une étoile dansante. Je vous le dis : vous portez encore un chaos en vous."
]

# configure a tokenizer
tokenizer = RegexpTokenizer('\w+')

# tokenization
doc_tokens = [
    [
        token.lower()
        for token in tokenizer.tokenize(doc)
        if token not in stopwords.words('french')
    ]
    for doc in corpus
]

doc_occurrences = [
    Counter(tokens)
    for tokens in doc_tokens
]

# tokens are the word-forms and
# vocabulary is an ordered list of unique tokens
vocabulary = sorted(set([
    token
    for doc in doc_tokens
    for token in doc
]))

# a null matrix
matrix = np.zeros((len(doc_tokens), len(vocabulary)))

# fill the matrix
for i, occurrences in enumerate(doc_occurrences):
    for j, word in enumerate(vocabulary):
        matrix[i, j] = occurrences[word]

df = pd.DataFrame(matrix, columns=vocabulary)

display(df)

## Une mesure TF-IDF à la main

Pour les besoins de l’exercice, vous ne vous concentrerez que sur le terme *devoirs* :

In [ ]:
display(df.devoirs)

### La fréquence du terme

La formule qui permet de calculer $\text{TF}$ est la suivante :

$$
\text{TF}(t, d) = \frac{t}{d}
$$

Sachant que $d$ est le nombre de tokens du vocabulaire dans le document, quelle est la mesure TF de *devoirs* dans le corpus ?

In [ ]:
# your code here

tfs = list()

for doc in doc_tokens:
    # number of words in the document
    n = len(doc)
    # occurrences of 'devoirs' set at 0
    c = 0
    # if 'devoirs' is found in the doc,
    # increase the count
    for token in doc:
        if token == "devoirs":
            c += 1
    tfs.append(c / n)

print(tfs)

### La fréquence inverse de document

La mesure IDF est régie par la formule ci-dessous :

$$
\text{IDF}(t) = \ln{\frac{N}{1 + \text{df}(t)}}
$$

Sachant que $N$ est le nombre total de documents dans le corpus et $\text{df}(t)$ le nombre de fois où le terme *devoirs* apparaît, quel est son IDF ?

In [ ]:
# your code here

N = len(doc_tokens)

d = sum([
    doc.count("devoirs")
    for doc in doc_tokens
])

idf = np.log(N / (1 + d))

print(idf)

### Calcul du TF-IDF

Il reste maintenant à appliquer la formule pour chaque document du corpus afin de s’apercevoir que le résultat est nul :

$$
\text{TF-IDF}(t, d) = \text{TF} \cdot \text{IDF}
$$

In [ ]:
# your code here

tfidfs = [
    tf * idf
    for tf in tfs
]

print(tfidfs)

Des scores négatifs signifient que le mot est sur-représenté dans les documents, mais notre interprétation doit être soumise à caution car le corpus est extrêmement restreint.

## Pondération de la matrice d’occurrences

In [ ]:
tf = df.div(df.sum(axis=1), axis=0)

In [ ]:
N = len(df)
df_t = (df > 0).sum(axis=0)
idf = ln(N / (1 + df_t))

In [ ]:
tfidf = tf * idf

In [ ]:
tfidf_df = pd.DataFrame(tfidf, columns=df.columns)

In [ ]:
tf.devoirs